<a href="https://colab.research.google.com/github/Nilaksh20/localrepo/blob/main/hacknu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle
from google.colab import files

files.upload()  # Upload kaggle.json when prompted

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"nilakshpathak1","key":"1369b60f4cfa52ac01f6117cbdf6c2bc"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d shubham2703/five-crop-diseases-dataset

Dataset URL: https://www.kaggle.com/datasets/shubham2703/five-crop-diseases-dataset
License(s): CC-BY-NC-SA-4.0


In [ ]:
import zipfile
z_r = zipfile.ZipFile('/content/five-crop-diseases-dataset.zip','r')
z_r.extractall()
z_r.close()


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, applications, optimizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import numpy as np

# Enable mixed precision and XLA compilation
tf.keras.mixed_precision.set_global_policy('mixed_float16')
tf.config.optimizer.set_jit(True)  # Enable XLA

# Configuration
img_height, img_width = 128, 128
batch_size = 32
dataset_path = "/content/Crop Diseases Dataset/Crop Diseases/Crop___Disease"

# First load dataset to get class names
temp_ds = keras.utils.image_dataset_from_directory(
    dataset_path,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=False
)
class_names = temp_ds.class_names
num_classes = len(class_names)
print("Class names:", class_names)

# Create actual train/val datasets
def create_dataset(subset):
    return keras.utils.image_dataset_from_directory(
        dataset_path,
        image_size=(img_height, img_width),
        batch_size=batch_size,
        validation_split=0.2,
        subset=subset,
        seed=42,
        label_mode='categorical'
    ).map(lambda x, y: (applications.mobilenet_v2.preprocess_input(x), y),
         num_parallel_calls=tf.data.AUTOTUNE)

train_ds = create_dataset("training")
val_ds = create_dataset("validation")

# Lightweight augmentation
def augment_images(x, y):
    x = tf.image.random_flip_left_right(x)
    x = tf.image.random_brightness(x, 0.1)
    return x, y

train_ds = train_ds.map(augment_images, num_parallel_calls=tf.data.AUTOTUNE)

# Optimize data pipeline
train_ds = train_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=tf.data.AUTOTUNE)

# Model construction
base_model = applications.MobileNetV2(
    input_shape=(img_height, img_width, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False

inputs = keras.Input(shape=(img_height, img_width, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(num_classes, activation='softmax', dtype='float32')(x)
model = keras.Model(inputs, outputs)

# Initial training
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5,
    callbacks=[
        EarlyStopping(monitor='val_loss', patience=3),
        ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2)
    ]
)

# Fine-tuning
base_model.trainable = True
for layer in base_model.layers[:-30]:
    layer.trainable = False

model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history_fine = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=3,
    callbacks=[EarlyStopping(monitor='val_loss', patience=2)]
)

# Evaluation
model.save("optimized_model.keras")
loss, accuracy = model.evaluate(val_ds)
print(f"Validation Accuracy: {accuracy*100:.2f}%")

Found 13324 files belonging to 5 classes.
Class names: ['Corn', 'Potato', 'Rice', 'Wheat', 'sugarcane']
Found 13324 files belonging to 5 classes.
Using 10660 files for training.
Found 13324 files belonging to 5 classes.
Using 2664 files for validation.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/5
334/334 ━━━━━━━━━━━━━━━━━━━━ 971s 3s/step - accuracy: 0.8845 - loss: 0.3451 - val_accuracy: 0.9917 - val_loss: 0.0295 - learning_rate: 0.0010
Epoch 2/5
334/334 ━━━━━━━━━━━━━━━━━━━━ 958s 3s/step - accuracy: 0.9934 - loss: 0.0255 - val_accuracy: 0.9947 - val_loss: 0.0167 - learning_rate: 0.0010
Epoch 3/5
334/334 ━━━━━━━━━━━━━━━━━━━━ 956s 3s/step - accuracy: 0.9965 - loss: 0.0143 - val_accuracy: 0.9955 - val_loss: 0.0141 - learning_rate: 0.0010
Epoch 4/5
334/334 ━━━━━━━━━━━━━━━━━━━━ 956s 3s/step - accuracy: 0.9965 - loss: 0.0115 - val_accuracy: 0.9955 - val_loss: 0.0122 - learning_rate: 0.0010
Epoch 5/5
334/334 ━━━━━━━━━━━━━━━━━━━━ 959s 3s/step - accuracy: 0.9973 - loss: 0.0092 - va

In [ ]:
predict_skin_disease("/content/m1.jpg")  # Change to your test image path

NameError: name 'predict_skin_disease' is not defined